In [ ]:
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
cmap = plt.get_cmap('jet_r')
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
from scipy.optimize import curve_fit
import folium
import branca
%matplotlib inline

In [ ]:
df_countries = pd.read_csv('../input/countries.csv')
df_countries['Region'] = df_countries['Region'].replace('ASIA', 'Asia')
df_countries['Region'] = df_countries['Region'].replace('AFRICA', 'Africa')
df_countries['Region'] = df_countries['Region'].replace('EUROPE', 'Europe')
df_countries['Region'] = df_countries['Region'].replace('LATIN', 'Latin')
df_countries['Region'] = df_countries['Region'].replace('NEAR_EAST', 'Near_East')
df_countries['Region'] = df_countries['Region'].replace('NORTHERN_AMERICA', 'Northern_America')
df_countries['Region'] = df_countries['Region'].replace('OCEANIA', 'Oceania')
df_table = pd.read_csv('../input/covid_19_clean_complete.csv', parse_dates=['Date'])
df_table['Active'] = df_table['Confirmed']-df_table['Deaths']-df_table['Recovered']
df_table['Country/Region'] = df_table['Country/Region'].replace('Mainland China', 'China')
df_table['Country/Region'] = df_table['Country/Region'].replace('US', 'United States')
df_table['Country/Region'] = df_table['Country/Region'].replace('UK', 'United Kingdom')
df_table['Country/Region'] = df_table['Country/Region'].replace('North Macedonia', 'Macedonia')
df_table['Country/Region'] = df_table['Country/Region'].replace('Bosnia and Herzegovina', 'Bosnia & Herzegovina')
df_table['Country/Region'] = df_table['Country/Region'].replace('South Africa', 'SouthAfrica')
df_table['Country/Region'] = df_table['Country/Region'].replace('Others', 'Ship')
df_table['Province/State'] = df_table['Province/State'].fillna('NA')
df_table.head()

In [ ]:
continent_dict = {}
for c in np.unique(df_countries['Country']):
    continent_dict[c] = df_countries[df_countries['Country']==c]['Region'].values[0]
continent_dict['Ship'] = 'None'
continent_dict['Saint Barthelemy'] = 'LATIN'
continent_dict['St. Martin'] = 'LATIN'
continent_dict['Palestine'] = 'NEAR_EAST'
continent_dict['Vatican City'] = 'EUROPE'
df_table['Continent'] = df_table['Country/Region'].apply(lambda x: continent_dict[x])

In [ ]:
df_latest = df_table[df_table['Date'] == max(df_table['Date'])].reset_index()
df_latest_grouped = df_latest.groupby('Country/Region')[
    'Confirmed', 'Deaths', 'Recovered'].sum().reset_index()

In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.plot(100*df_table[['Date', 'Confirmed']][df_table['Country/Region']=='China'].groupby(
    'Date').sum()/df_table[['Date', 'Confirmed']].groupby('Date').sum(), '-b', linewidth=4, label='Confirmed')
plt.plot(100*df_table[['Date', 'Deaths']][df_table['Country/Region']=='China'].groupby(
    'Date').sum()/df_table[['Date', 'Deaths']].groupby('Date').sum(), '-r', linewidth=4, label='Death')
plt.grid()
plt.xlabel('Date')
plt.xticks(rotation=20)
plt.ylabel('China / World percentage')
plt.ylim([70, 101])
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(100*df_table[['Date', 'Confirmed']][df_table['Continent']=='Asia'].groupby(
    'Date').sum()/df_table[['Date', 'Confirmed']].groupby('Date').sum(), '-b', linewidth=4, label='Confirmed')
plt.plot(100*df_table[['Date', 'Deaths']][df_table['Continent']=='Asia'].groupby(
    'Date').sum()/df_table[['Date', 'Deaths']].groupby('Date').sum(), '-r', linewidth=4, label='Death')
plt.grid()
plt.xlabel('Date')
plt.xticks(rotation=20)
plt.ylabel('Asia / World percentage')
plt.ylim([70, 101])
plt.legend()
plt.savefig('../figs/percent_world.png')

In [ ]:
plt.figure(figsize=(8, 6))
n = 3.
plt.plot(df_table[['Date', 'Deaths', 'Confirmed']].groupby('Date').sum().apply(
    lambda x: 100.0-100.0/(1+np.float64(x['Deaths'])/np.float64(x['Confirmed'])), axis=1),
         '-b', linewidth=4, c=cmap(np.arange(0, 1, 1./n))[0], label='World')
plt.plot(df_table[df_table['Country/Region']=='China'][['Date', 'Deaths', 'Confirmed']].groupby('Date').sum().apply(
    lambda x: 100.0-100.0/(1+np.float64(x['Deaths'])/np.float64(x['Confirmed'])), axis=1), 
         '-r', linewidth=4, c=cmap(np.arange(0, 1, 1./n))[1], label='China')
plt.plot(df_table[df_table['Country/Region']=='Italy'][['Date', 'Deaths', 'Confirmed']].groupby('Date').sum().apply(
    lambda x: 100.0-100.0/(1+np.float64(x['Deaths'])/np.float64(x['Confirmed'])), axis=1), 
         '-g', linewidth=4, c=cmap(np.arange(0, 1, 1./n))[2], label='Italy')
#plt.plot(df_table[df_table['Country/Region']=='Iran'][['Date', 'Deaths', 'Confirmed']].groupby('Date').sum().apply(
#    lambda x: 100.0-100.0/(1+np.float64(x['Deaths'])/np.float64(x['Confirmed'])), axis=1), 
#         '-g', linewidth=4, c=cmap(np.arange(0, 1, 1./n))[3], label='Iran')
#plt.plot(df_table[df_table['Country/Region']=='Germany'][['Date', 'Deaths', 'Confirmed']].groupby('Date').sum().apply(
#    lambda x: 100.0-100.0/(1+np.float64(x['Deaths'])/np.float64(x['Confirmed'])), axis=1), 
#         '-g', linewidth=4, c=cmap(np.arange(0, 1, 1./n))[4], label='Germany')
#plt.plot(df_table[df_table['Country/Region']=='United States'][['Date', 'Deaths', 'Confirmed']].groupby('Date').sum().apply(
#    lambda x: 100.0-100.0/(1+np.float64(x['Deaths'])/np.float64(x['Confirmed'])), axis=1), 
#         '-g', linewidth=4, c=cmap(np.arange(0, 1, 1./n))[5], label='United Staets')
plt.xlabel('Date')
plt.xticks(rotation=20)
plt.ylabel('Death Rate [%]')
plt.grid()
plt.ylim([-0,6.0])
plt.legend()
plt.savefig('../figs/death_percentage.png')

In [ ]:
plt.figure(figsize=(8, 6))

cols = ['Confirmed', 'Deaths', 'Recovered', 'Active']
country = 'China'
continent = 'Asia'
for ic, col in enumerate(cols):
    plt.plot(df_table.groupby('Date').sum()[col], '-', 
             c=cmap(np.arange(0, 1, 1./len(cols)))[ic], linewidth=4, label='World '+col)
    plt.plot(df_table[df_table['Country/Region']==country].groupby('Date').sum()[
        col], '--', c=cmap(np.arange(0, 1, 1./len(cols)))[ic], linewidth=3, label=country+' '+col)
    plt.plot(df_table[df_table['Continent']==continent].groupby('Date').sum()[
        col], ':', c=cmap(np.arange(0, 1, 1./len(cols)))[ic], linewidth=3, label=continent+' '+col)

plt.title('World')
plt.xlabel('Date')
plt.xticks(rotation=20)
plt.ylabel('Worldwide Cases')
#plt.yscale('log')
plt.grid()
plt.legend()
plt.savefig('../figs/world_cases.png')

In [ ]:
plt.figure(figsize=(8, 6))

c = 'Italy'
cols = ['Confirmed', 'Deaths', 'Recovered', 'Active']
t = df_table[df_table['Country/Region']==c][['Date']]
y = df_table[df_table['Country/Region']==c][cols]
litaly = plt.plot(t, y, '-', linewidth=4)

plt.title(c)
plt.xlabel('Date')
plt.xticks(rotation=20)
plt.ylabel('Confirmed Cases')
plt.yscale('log')
plt.grid()
plt.legend(litaly, cols)

In [ ]:
plt.figure(figsize=(8, 6))

c = 'United States'
cols = ['Confirmed', 'Deaths', 'Recovered', 'Active']
lus = plt.plot(df_table[df_table['Country/Region']==c].groupby('Date').sum()[
    cols], linewidth=4)

plt.title(c)
plt.xlabel('Date')
plt.xticks(rotation=20)
plt.ylabel('Worldwide Cases')
plt.yscale('log')
plt.grid()
plt.legend(lus, cols)

In [ ]:
#temp = df.groupby(['Country/Region', 'Province/State'])['Confirmed', 'Deaths', 'Recovered'].max()
#temp.style.background_gradient(cmap='Pastel1_r')

In [ ]:
#temp = df.groupby(['Country/Region', 'Province/State'])['Confirmed', 'Deaths', 'Recovered'].max().groupby(['Country/Region']).sum()
#temp = temp.sort_values(by='Confirmed', ascending=False)
#temp.style.background_gradient(cmap='Pastel1_r')

In [ ]:
#temp = df.groupby(['Country/Region', 'Province/State'])['Confirmed', 'Deaths', 'Recovered'].max().groupby(['Country/Region']).sum()
#temp = temp.sort_values(by='Deaths', ascending=False)
#temp.style.background_gradient(cmap='Pastel1_r')

In [ ]:
#temp = df.groupby('Date')['Confirmed', 'Deaths', 'Recovered'].sum()
#temp.tail(1)

In [ ]:
#df_full = df.groupby(['Country/Region', 'Province/State']).agg({
#    'Confirmed':'max', 'Deaths':'max', 'Recovered':'max','Lat':'mean', 'Long':'mean'})

In [ ]:
r = 100+(np.log10(df_latest['Confirmed']))*2e4
colorscale = branca.colormap.linear.YlOrRd_09.scale(0, np.max(r.values))

In [ ]:
m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=10, zoom_start=1)
for i in range(df_latest.shape[0]):
    folium.Circle(location=[df_latest.iloc[i]['Lat'], df_latest.iloc[i]['Long']],
                  radius=r[i],
                  tooltip = '<li><bold>Country: ' + df_latest.iloc[i]['Country/Region'] + '</bold</li>' + 
                  '<li><bold>Province: ' + df_latest.iloc[i]['Province/State'] + '</bold</li>' + 
                  '<li><bold>Confirmed: ' + str(df_latest.iloc[i]['Confirmed']) + '</bold</li>' + 
                  '<li><bold>Deaths: ' + str(df_latest.iloc[i]['Deaths']) + '</bold</li>' + 
                  '<li><bold>Recovered: ' + str(df_latest.iloc[i]['Recovered']) + '</bold</li>'
                 ).add_to(m)
m